## UTide analysis of ADCP data

In [1]:
%matplotlib ipympl

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import gsw
import matplotlib.dates as mdates
import utide

import datapath

In [2]:
ds = xr.open_dataset(datapath.adcpnc())

In [3]:
height = ds['binheight'] + 0.4572 # add instrument height (18 inches)

time = mdates.date2num(ds['time'])

coef = utide.solve(time, ds['Eas'][:,0], ds['Nor'][:,0],
         lat=35+8.4585/60,
         method='robust')

constit_list = coef['name']

Lsmaj = np.nan*np.ones([len(height),len(constit_list)])
Lsmin = np.nan*np.ones([len(height),len(constit_list)])
theta = np.nan*np.ones([len(height),len(constit_list)])
g = np.nan*np.ones([len(height),len(constit_list)])
SNR = np.nan*np.ones([len(height),len(constit_list)])
PE = np.nan*np.ones([len(height),len(constit_list)])

Lsmaj_ci = np.nan*np.ones([len(height),len(constit_list)])
Lsmin_ci = np.nan*np.ones([len(height),len(constit_list)])
theta_ci = np.nan*np.ones([len(height),len(constit_list)])
g_ci = np.nan*np.ones([len(height),len(constit_list)])

# tidal analysis
time = mdates.date2num(ds['time'])
for zi in np.arange(len(ds['binheight'])):
    print('bin '+str(zi+1)+'/'+str(len(ds['binheight'])))
    coef = utide.solve(time, ds['Eas'][:,zi], ds['Nor'][:,zi],
             lat=35+8.4585/60,
             method='robust')
    
    ni = np.argsort(coef['aux']['frq'])
    
    Lsmaj[zi,:] = coef['Lsmaj'][ni]
    Lsmin[zi,:] = coef['Lsmin'][ni]
    theta[zi,:] = coef['theta'][ni]
    g[zi,:] = coef['g'][ni]
    SNR[zi,:] = coef['diagn']['SNR'][ni]
    PE[zi,:] = coef['diagn']['PE'][ni]
    Lsmaj_ci[zi,:] = coef['Lsmaj_ci'][ni]
    Lsmin_ci[zi,:] = coef['Lsmin_ci'][ni]
    theta_ci[zi,:] = coef['theta_ci'][ni]
    g_ci[zi,:] = coef['g_ci'][ni]

solve: matrix prep ... solution ... diagnostics ... done.
bin 1/23
solve: matrix prep ... solution ... diagnostics ... done.
bin 2/23
solve: matrix prep ... solution ... diagnostics ... done.
bin 3/23
solve: matrix prep ... solution ... diagnostics ... done.
bin 4/23
solve: matrix prep ... solution ... diagnostics ... done.
bin 5/23
solve: matrix prep ... solution ... diagnostics ... done.
bin 6/23
solve: matrix prep ... solution ... diagnostics ... done.
bin 7/23
solve: matrix prep ... solution ... diagnostics ... done.
bin 8/23
solve: matrix prep ... solution ... diagnostics ... done.
bin 9/23
solve: matrix prep ... solution ... diagnostics ... done.
bin 10/23
solve: matrix prep ... solution ... diagnostics ... done.
bin 11/23
solve: matrix prep ... solution ... diagnostics ... done.
bin 12/23
solve: matrix prep ... solution ... diagnostics ... done.
bin 13/23
solve: matrix prep ... solution ... diagnostics ... done.
bin 14/23
solve: matrix prep ... solution ... diagnostics ... done.

In [4]:
dstide = xr.Dataset({'Lsmaj': (('height', 'constituent'), Lsmaj),
                     'Lsmin': (('height', 'constituent'), Lsmin),
                     'theta': (('height', 'constituent'), theta),
                     'g': (('height', 'constituent'), g),
                     'SNR': (('height', 'constituent'), SNR),
                     'PE': (('height', 'constituent'), PE),
                     'Lsmaj_ci': (('height', 'constituent'), Lsmaj_ci),
                     'Lsmin_ci': (('height', 'constituent'), Lsmin_ci),
                     'theta_ci': (('height', 'constituent'), theta_ci),
                     'g_ci': (('height', 'constituent'), g_ci),
                     'freq':(('constituent'),coef['aux']['frq'][ni])},
                {'height': np.array(height), 'constituent': coef['name'][ni]})

In [5]:
dstide.attrs['reftime']= coef['aux']['reftime']

In [6]:
dstide.to_netcdf('data/utide_results.nc')